# Experiment 3: Acquiring time-domain IQ data at a fixed flux point with varying `(f_clearing, P_clearing)` for varying readout powers (interleaved with no clearing tone)

**Goal:**

Get `release_rates(var_clearing_tone_freq, var_clearing_tone_power, varying_readout_power, FIXED_PHI)`

**Method:**

- Fix phi, find f(phi), set drive tone to f(phi) - detuning
- Vary clearing tone and clearing power and record I,Q data
- Turn off clearing tone and record I,Q data
- Run HMM analysis on the recorded data to get the rates

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline

In [ ]:
from flux_fit import *
from utils import *

In [ ]:
!python -m srsinst.dc205

## Setting up the Experiment


### Variables and Constants

In [ ]:
project_name = "Andreev_Spectroscopy"
device_name = "L1A"
run_id = 2
T_MXC = 24
T_Rad = -1
experiment_name = f"{device_name}_RUN{run_id}"

drive_path = r"G:\Shared drives\LFL\Projects\Quasiparticles"
data_path = set_project(r"{}\{}".format(drive_path, project_name), f"{experiment_name}")

# Constants
SPATH = data_path
FIG_PATH = os.path.join(SPATH, "Figures")
LOG_FILE = os.path.join(SPATH, f'MEASUREMENTLOG_{time.strftime("%Y%m%d_%H%M%S")}.log')
PATH_TO_EXE = r'C:/Users/LFL/lflPython/AlazarDrivers/CS_Average/x64/Release/ATS9371_CS_Average.exe'

# Ensure directories exist
os.makedirs(FIG_PATH, exist_ok=True)

# Setup logging
logging.basicConfig(filename=LOG_FILE, filemode='w', level=logging.INFO)

### Specify the bias point parameter for the experiment


Setting up the phi's and getting the required voltage array with the detuning for the drive tone.

In [ ]:
phi_value, voltage_in_V = 0.49, 0.005

### Instrument Setup

In [ ]:
client = Labber.connectToServer(timeout=None)
instrument_list = client.getListOfInstruments()
instruments = create_instrument_connections(client, instrument_list)

The labber instrument server config is saved as a hdf5 file in this repository. Instantiating the instruments we need now.

In [ ]:
vs = connect_SRS()

VNA = instruments['Agilent Network Analyzer E5071B']
DA = instruments['Vaunix Lab Brick Digital Attenuator']
SMU = instruments['Keithley 2400 SourceMeter']
LO= instruments['SignalCore SC5511A Signal Generator_10002F25_LO']
TWPA_PUMP= instruments['SignalCore SC5511A Signal Generator_1000334A_TWPA']
Drive = instruments['Rohde&Schwarz RF Source']

initialize_instruments(vna=VNA, da=DA, smu=SMU, lo=LO, drive=Drive, srs=vs, twpa_pump=TWPA_PUMP)

In [ ]:
vs.check_id()

Manually setting the VNA to the correct setting.

In [ ]:
# VNA + LO Drive
span = VNA.getValue('Span') # Hz
electrical_delay = VNA.getValue('Electrical Delay') # s
vna_power = 5 # dBm
vna_avg = 20

print(f"VNA Span: {span} Hz, Electrical Delay: {electrical_delay} s")

Set the clearing pump parameters

In [ ]:
# Clearing
f_low, f_high = 9.5, 13 # GHz
f_step = 0.5 # GHz
f_clearing_arr = np.arange(f_low, f_high+f_step, f_step)
print(f"f_clearing_arr: {f_clearing_arr}")

P_low, P_high = -25, -5
P_step = 1
P_clearing_arr = np.arange(P_low, P_high+P_step, P_step)
print(f"P_clearing_arr: {P_clearing_arr}")

Set the digitizer parameters

In [ ]:
num_traces=1
acquisitionLength_sec=30
origRateMHz=300
sampleRateMHz=10
averageTimeCycle=0

Set the DA attenuation to vary the readout power

In [ ]:
lowerBound=30 # dB for DA attenuator
upperBound=30 # dB for DA attenuator

## The driver code

In [ ]:
# Create a log file for the VNA data
lfVNA = Labber.createLogFile_ForData(os.path.join(SPATH, f'{device_name}_{project_name}_run-{run_id}_clearing_tone_spectroscopy'),
                                      [{'name':'VNA - S21','complex':True,'vector':True,'x_name':'Frequency','x_unit':'Hz'}],
                                      step_channels=[
                                          {'name':'Clearing Tone Power','values':P_clearing_arr.tolist(),'unit':'dBm'},
                                          {'name':'Clearing Tone Frequency','values':f_clearing_arr.tolist(),'unit':'GHz'},
                                          ]
                                      )

initialize_logging(lfVNA, SPATH, PATH_TO_EXE, FIG_PATH, experiment_name, device_name)

We have checked in advance that the TWPA pump with the following configuration gives us +20 dB in the entire frequency range of interest.

In [ ]:
set_TWPA_pump(f=5.946, power=5.88)

## Running the experiment and saving the data to the external drive:

Set the flux bias

In [ ]:
print("="*50)
voltage = voltage_in_V # V
phi = phi_value # flux quanta
print(f"phi: {phi:.6f}, voltage: {voltage:.6f} V")
set_flux_bias_srs_in_V(voltage, step=0.001)

Set the LO for the digitizer

In [ ]:
f_drive=5.722890e9 # Hz
f_phi=5.723113e9 # Hz
f_d_power = 16 # dBm

In [ ]:
detuning = f_drive-f_phi # Hz

In [ ]:
start_time = time.time()

turn_off_clearing()
turn_off_vna()

set_LO_tone(f_drive, f_d_power)
print(f"f_phi: {f_phi*1e-9:.6f} GHz, f_drive: {f_drive*1e-9:.6f} GHz") 

for f_clearing in f_clearing_arr:
    
    # record data with no clearing tone for reference
    turn_off_clearing()
    print(f"Taking reference data at phi = {phi:.3f} while driving at {f_drive:.6f} GHz")
    logging.info(f'\nTaking reference data at phi = {phi:.3f} while driving at {f_drive:.6f} GHz')
    ref_files = acquire_IQ_data(phi, f_clearing, None, num_traces, acquisitionLength_sec, origRateMHz, sampleRateMHz, averageTimeCycle, lowerBound, upperBound)
    # Update the metadata files with the current parameters
    for ref_file in ref_files:
            write_metadata(ref_file, acquisitionLength_sec, origRateMHz, f_drive, voltage, T_MXC, T_Rad, phi, f_clearing, None)
    
    for P_clearing in P_clearing_arr:
        
        print(f"f_clearing: {f_clearing:.6f} GHz, P_clearing: {P_clearing:.6f} dBm")
        set_clearing_tone(f_clearing, P_clearing)
        logging.info(f'\nStarting Alazar acquisition at phi = {phi:.3f} while driving at {f_drive:.6f} GHz')
        metadata_files = acquire_IQ_data(phi, f_clearing, P_clearing, num_traces, acquisitionLength_sec, origRateMHz, sampleRateMHz, averageTimeCycle, lowerBound, upperBound)
        
        # Update the metadata files with the current parameters
        for metadata_file in metadata_files:
            write_metadata(metadata_file, acquisitionLength_sec, origRateMHz, f_drive, voltage, T_MXC, T_Rad, phi, f_clearing, P_clearing)
            
        # Save the VNA data
        turn_off_LO()
        turn_on_vna()
        # grab the VNA trace
        dData = VNA.getValue('S21')
        zData = dData['y']
        xBG = np.arange(dData['t0'],dData['t0']+dData['shape'][0]*dData['dt'],dData['dt'])
        # Save the VNA data
        td2 = Labber.getTraceDict(zData,x0=xBG[0],x1=xBG[-1])
        lfVNA.addEntry({'VNA - S21':td2})
        # Turn off the VNA
        turn_off_vna()
        turn_on_LO()
         
        sleep(0.1)
        print(f'Finished loop for one datasweep with clearing tone ON in {time.time() - start_time:.2f} seconds')
        logging.info(f'Finished loop for one datasweep with clearing tone ON in {time.time() - start_time:.2f} seconds')


# Close the log file
print(f'Finished the experiment for {device_name} in {time.time() - start_time:.2f} seconds')
logging.info(f'Finished the experiment for {device_name} in {time.time() - start_time:.2f} seconds')

# Test Cell

In [ ]:
start_time = time.time()

turn_off_clearing()
turn_off_vna()

set_LO_tone(f_drive, f_d_power)
print(f"f_phi: {f_phi*1e-9:.6f} GHz, f_drive: {f_drive*1e-9:.6f} GHz") 

for f_clearing in f_clearing_arr:
    
    # record data with no clearing tone for reference
    turn_off_clearing()
    print(f"Taking reference data at phi = {phi:.3f} while driving at {f_drive:.6f} GHz")
    logging.info(f'\nTaking reference data at phi = {phi:.3f} while driving at {f_drive:.6f} GHz')
    """
    metadata_files = acquire_IQ_data(phi, f_clearing, None, num_traces, acquisitionLength_sec, origRateMHz, sampleRateMHz, averageTimeCycle, lowerBound, upperBound)
    # Update the metadata files with the current parameters
    for metadata_file in metadata_files:
            write_metadata(metadata_file, acquisitionLength_sec, origRateMHz, f_drive, voltage, T_MXC, T_Rad, phi, f_clearing, None)
    """
    for P_clearing in P_clearing_arr:
        
        print(f"f_clearing: {f_clearing:.6f} GHz, P_clearing: {P_clearing:.6f} dBm")
        set_clearing_tone(f_clearing, P_clearing)
        logging.info(f'\nStarting Alazar acquisition at phi = {phi:.3f} while driving at {f_drive:.6f} GHz')
        """
        metadata_files = acquire_IQ_data(phi, f_clearing, P_clearing, num_traces, acquisitionLength_sec, origRateMHz, sampleRateMHz, averageTimeCycle, lowerBound, upperBound)
        
        # Update the metadata files with the current parameters
        for metadata_file in metadata_files:
            write_metadata(metadata_file, acquisitionLength_sec, origRateMHz, f_drive, voltage, T_MXC, T_Rad, phi, f_clearing, P_clearing)
        """
        # Save the VNA data
        turn_off_LO()
        turn_on_vna()
        # grab the VNA trace
        dData = VNA.getValue('S21')
        zData = dData['y']
        xBG = np.arange(dData['t0'],dData['t0']+dData['shape'][0]*dData['dt'],dData['dt'])
        # Save the VNA data
        td2 = Labber.getTraceDict(zData,x0=xBG[0],x1=xBG[-1])
        lfVNA.addEntry({'VNA - S21':td2})
        # Turn off the VNA
        turn_off_vna()
        turn_on_LO()
         
        sleep(0.1)
        print(f'Finished loop for one datasweep with clearing tone ON in {time.time() - start_time:.2f} seconds')
        logging.info(f'Finished loop for one datasweep with clearing tone ON in {time.time() - start_time:.2f} seconds')


# Close the log file
print(f'Finished the experiment for {device_name} in {time.time() - start_time:.2f} seconds')
logging.info(f'Finished the experiment for {device_name} in {time.time() - start_time:.2f} seconds')